In [1]:
import numpy as np
from scipy.special import logit,expit
import collections
import matplotlib.pyplot as plt

In [2]:
def g(x,y):
    y=np.dot(x,y)
    return y

def f(x,y):
    y=np.dot(x,y)
    return y

def pi(x,y):
    y=expit(g(x,y))
    return y

def phi(x,y):
    y=expit(f(x,y))
    return y

def I0(y):
    if y==0:
        return 1

In [3]:
#E-step
def Lih(X,beta):
    Ga=np.zeros((N,2))

    for i in range(N):#响应度
        if Y[i]==1:
            Ga[i,0]=0
            Ga[i,1]=1
        elif Y[i]==0:
            a=(expit(np.dot(X[i],beta[0])))
            b=(1-a)*(1-expit(np.dot(X[i],beta[1])))
            Ga[i,0]=a/(a+b)
            Ga[i,1]=1-Ga[i,0]

    return Ga


#M-step
def Q(X,Y,beta):
    Ga=Lih(X,beta)
    A=np.zeros(N)
    for i in range(N):
        pi=expit(np.dot(beta[0],X[i]))
        phi=expit(np.dot(beta[1],X[i]))
        if Y[i]==0:
            A[i]=(Ga[i,0])*np.log(pi)+(Ga[i,1])*np.log((1-pi)*(1-phi))
            #A[i]=Ga[i,0]*np.log(pi)+Ga[i,1]*np.log((1-pi)*(1-phi))
            #A[i]=(Ga[i,0]*np.log(pi))+(Ga[i,1]*(np.log(1-pi)+np.log(1-phi)))
        else:
            A[i]=(Ga[i,1])*np.log((1-pi)*(phi))
            #A[i]=Ga[i,0]*np.log(pi)+Ga[i,1]*np.log((1-pi)*(1-phi))
            #A[i]=(Ga[i,0]*np.log(pi))+(Ga[i,1]*(np.log(1-pi)+np.log(phi)))
        #A[i]=(Ga[i,0]*(1-Y[i])*np.log(pi))+(Ga[i,1]*(np.log(1-pi)+np.log(1-phi)+Y[i]*np.dot(X[i],beta[1])))
    Q=np.sum(A)
    return Q

def grad(X,Y,Ga,beta):
    A=np.zeros((N,P))
    B=np.zeros((N,P))
    for i in range(N):
        A[i]=((Ga[i,0]*(1-Y[i]))*(1-expit(np.dot(beta[0],X[i])))-(Ga[i,1]*expit(np.dot(beta[0],X[i]))))*X[i]
        B[i]=(Ga[i,1]*(Y[i]-expit(np.dot(beta[1],X[i]))))*X[i]
    grad1=np.sum(A,axis=0)
    grad2=np.sum(B,axis=0)
    grad=np.array([grad1,grad2])
    return grad


#EM
def dgra(X,Y,ibeta,n=1000,t=0.001,xim=0.99,yim=0.1):#梯度下降（回溯梯度下降法）xim为步长缩小倍数

    for i in range(n):#M-step
        lik1=Q(X,Y,ibeta)
        ibeta1=ibeta
        L=Lih(X,ibeta)#E-step
        ibeta=ibeta+t*grad(X,Y,L,ibeta)
        lik=Q(X,Y,ibeta)
        d=np.r_[ibeta[0],ibeta[1]]
        c=np.dot(d,d)
        b=(lik-lik1)
        print('i=',i,'b=',b,'\tstep=',t)
        print('Q=',Q(X,Y,ibeta))
        if b>=yim*t*c :
            t=xim*t
        else:
            break
    return ibeta


In [4]:
N=500
M=50
P=3
np.random.seed(123)


In [5]:
def creatB():
    b1=0.1*np.random.rand(3)-0.05
    b2=np.random.rand(3)-0.5
    b1[2]=-0.5*np.random.rand()-0.5
    b2[1]=0.5*np.random.rand()+0.5
    b=np.array([b1,b2])
    return b

def creatX():#X[i,j]表示第i个样本第j维分量
    X=np.zeros((N,3))
    for i in range(N):
        a=np.zeros(3)
        mu=[-1.5,-1.5]
        cov=np.array([[1,0.3],[0.3,1]])
        a[0]=1
        a[1:]=np.random.multivariate_normal(mu,cov,1,check_valid='raise')
        X[i]=a
    return X


def creatY(X,beta):
    Y=np.zeros(N)
    for i in range(N):
        a=np.random.rand()
        b=pi(X[i],beta[0])
        if a>=b:
            c=np.random.rand()
            d=phi(X[i],beta[1])
            if c>=d:
                Y[i]=1
    return Y

In [6]:
Y=np.zeros(N)
beta=creatB()
A=creatX()
B=expit(np.dot(A,beta.T))
C=np.random.rand(N,2)
for j in range(N):
    if C[j,0]>B[j,0]:
        if C[j,1]<B[j,1]:
            Y[j]=1

In [7]:
np.array(np.where(Y[:]==1)).shape

(1, 29)

In [11]:
X=np.zeros((M,N,3))
Y=np.zeros((M,N))
Betat=np.zeros((M,2,3))
Corr=np.zeros(M)
for i in range(M):
    beta=creatB()
    Betat[i]=beta
    A=creatX()
    X[i]=A
    B=expit(np.dot(A,beta.T))
    C=np.random.rand(N,2)
    for j in range(N):
        if C[j,0]>B[j,0]:
            if C[j,1]<B[j,1]:
                Y[i,j]=1
    Corr[i]=np.array(np.where(Y[i]==1)).shape[1]/N